In [1]:
# #Splitting the data to train, test and alidation
# import os
# import shutil
# import random

# # Define the source directory where the images are found
# source_dir = r"D:\Data_Science_Projects\PlantVillage"

# # Define the base directory where the organized folders will be saved
# base_dir = r"D:\Data_Science_Projects\PlantVillage2"

# # Define the subdirectories containing the images
# subdirectories = [
#     'Pepper__bell___Bacterial_spot',
#     'Pepper__bell___healthy',
#     'Potato___Early_blight',
#     'Potato___healthy',
#     'Potato___Late_blight',
#     'Tomato_Bacterial_spot',
#     'Tomato_Early_blight',
#     'Tomato_healthy',
#     'Tomato_Late_blight',
#     'Tomato_Leaf_Mold',
#     'Tomato_Septoria_leaf_spot',
#     'Tomato_Spider_mites_Two_spotted_spider_mite',
#     'Tomato__Target_Spot',
#     'Tomato__Tomato_mosaic_virus',
#     'Tomato__Tomato_YellowLeaf__Curl_Virus'
# ]

# # Create the train, validation, and test directories if they do not exist
# os.makedirs(os.path.join(base_dir, 'train'), exist_ok=True)
# os.makedirs(os.path.join(base_dir, 'validation'), exist_ok=True)
# os.makedirs(os.path.join(base_dir, 'test'), exist_ok=True)

# # Define the distribution ratios
# train_ratio = 0.7
# validation_ratio = 0.15
# test_ratio = 0.15

# # Iterate through each subdirectory
# for subdirectory in subdirectories:
#     source_path = os.path.join(source_dir, subdirectory)

#     # List all files in the source directory
#     files = os.listdir(source_path)

#     # Calculate the number of files for each split
#     total_files = len(files)
#     train_count = int(train_ratio * total_files)
#     validation_count = int(validation_ratio * total_files)

#     # Shuffle files and split into train, validation, and test sets
#     shuffled_files = files[:]  # Create a copy of the file list
#     random.shuffle(shuffled_files)  # Shuffle the list of files

#     # Create subdirectories for each category in the base directory
#     os.makedirs(os.path.join(base_dir, 'train', subdirectory), exist_ok=True)
#     os.makedirs(os.path.join(base_dir, 'validation', subdirectory), exist_ok=True)
#     os.makedirs(os.path.join(base_dir, 'test', subdirectory), exist_ok=True)

#     # Move files to respective folders
#     for i, file in enumerate(shuffled_files):
#         if i < train_count:
#             shutil.move(os.path.join(source_path, file), os.path.join(base_dir, 'train', subdirectory, file))
#         elif i < train_count + validation_count:
#             shutil.move(os.path.join(source_path, file), os.path.join(base_dir, 'validation', subdirectory, file))
#         else:
#             shutil.move(os.path.join(source_path, file), os.path.join(base_dir, 'test', subdirectory, file))

# print("Data has been organized into train, validation, and test folders.")


Data Preprocessing

In [2]:
import os
from PIL import Image

# Define the base directory and target size
base_dir = r"D:\Data_Science_Projects\PlantVillage2"
target_size = (224, 224)  # Define your target size

# Function to resize images
def resize_images(base_directory):
    for dataset in ['train', 'validation', 'test']:
        directory = os.path.join(base_directory, dataset)
        for subdir, _, files in os.walk(directory):
            for file in files:
                file_path = os.path.join(subdir, file)
                with Image.open(file_path) as img:
                    img = img.resize(target_size)
                    img.save(file_path)  # Overwrite the original image

# Resize images in all directories
resize_images(base_dir)
print("Images have been resized.")


Images have been resized.


In [3]:
#Normalization of images
import numpy as np

# Function to normalize images
def normalize_images(base_directory):
    for dataset in ['train', 'validation', 'test']:
        directory = os.path.join(base_directory, dataset)
        for subdir, _, files in os.walk(directory):
            for file in files:
                file_path = os.path.join(subdir, file)
                with Image.open(file_path) as img:
                    img_array = np.array(img) / 255.0  # Normalize pixel values
                    img_array = (img_array * 255).astype(np.uint8)  # Convert back to uint8
                    img = Image.fromarray(img_array)
                    img.save(file_path)  # Overwrite the original image

# Normalize images in all directories
normalize_images(base_dir)
print("Images have been normalized.")


Images have been normalized.


In [4]:
#Data Augmentation
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Function to perform data augmentation
def augment_images(base_directory, save_to_directory):
    train_dir = os.path.join(base_directory, 'train')
    for subdir, _, files in os.walk(train_dir):
        class_name = os.path.basename(subdir)
        os.makedirs(os.path.join(save_to_directory, class_name), exist_ok=True)  # Create class folder

        for file in files:
            file_path = os.path.join(subdir, file)
            img = tf.keras.preprocessing.image.load_img(file_path)
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)

            # Generate augmented images
            i = 0
            for batch in datagen.flow(img_array, batch_size=1, save_to_dir=os.path.join(save_to_directory, class_name), save_prefix='aug', save_format='jpeg'):
                i += 1
                if i >= 5:  # Limit to 5 augmented images per original
                    break

# Create a directory for augmented images
augmented_dir = r"D:\Data_Science_Projects\PlantVillage2\augmented"
augment_images(base_dir, augmented_dir)
print("Data augmentation for training set completed.")


Data augmentation for training set completed.


Looping through various models to find the best

In [8]:
import os
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, EfficientNetB0, MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models


In [3]:
#Data Preprocessing
# Directories
base_dir = r"D:\Data_Science_Projects\PlantVillage2"
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1.0/255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 14441 images belonging to 15 classes.
Found 3088 images belonging to 15 classes.


In [4]:
#Function to Build and Compile Models
def build_model(model_name, input_shape=(224, 224, 3), num_classes=10):
    if model_name == 'CNN':
        model = Sequential([
            tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            Flatten(),
            Dense(256, activation='relu'),
            Dense(num_classes, activation='softmax')
        ])
    elif model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
        model = Sequential([base_model, GlobalAveragePooling2D(), Dense(256, activation='relu'), Dense(num_classes, activation='softmax')])
    elif model_name == 'VGG19':
        base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
        model = Sequential([base_model, GlobalAveragePooling2D(), Dense(256, activation='relu'), Dense(num_classes, activation='softmax')])
    elif model_name == 'ResNet':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
        model = Sequential([base_model, GlobalAveragePooling2D(), Dense(256, activation='relu'), Dense(num_classes, activation='softmax')])
    elif model_name == 'EfficientNet':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
        model = Sequential([base_model, GlobalAveragePooling2D(), Dense(256, activation='relu'), Dense(num_classes, activation='softmax')])
    elif model_name == 'MobileNet':
        base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)
        model = Sequential([base_model, GlobalAveragePooling2D(), Dense(256, activation='relu'), Dense(num_classes, activation='softmax')])
    
    # Compile the model
    model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [5]:
#Defining a Function to Calculate Metrics
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    cm = confusion_matrix(y_true, y_pred)
    
    return accuracy, precision, recall, f1, cm


Tried the following models:

CNN
VGG16/19
ResNet
EfficientNet
MobileNet


In [ ]:
# # Train and Evaluate Models in a Loop
# from sklearn.preprocessing import LabelEncoder


# models = ['MobileNet', 'CNN']
# best_model = None
# best_f1_score = 0

# # Store results for each model
# results = {}

# # Loop through each model
# for model_name in models:
#     print(f"\nTraining {model_name} model...")
    
#     # Build the model
#     model = build_model(model_name, num_classes=train_generator.num_classes)
    
#     # Train the model
#     model.fit(
#         train_generator,
#         steps_per_epoch=train_generator.samples // train_generator.batch_size,
#         validation_data=validation_generator,
#         validation_steps=validation_generator.samples // validation_generator.batch_size,
#         epochs=3  # You can increase this if needed
#     )
    
#     # Predict on validation data
#     validation_generator.reset()
#     predictions = model.predict(validation_generator)
#     y_pred = np.argmax(predictions, axis=1)
    
#     # True labels
#     y_true = validation_generator.classes
    
#     # Calculate metrics
#     accuracy, precision, recall, f1, cm = calculate_metrics(y_true, y_pred)
    
#     # Print the metrics
#     print(f"\nResults for {model_name}:")
#     print(f"Accuracy: {accuracy * 100:.2f}%")
#     print(f"Precision: {precision:.2f}")
#     print(f"Recall: {recall:.2f}")
#     print(f"F1 Score: {f1:.2f}")
#     print(f"Confusion Matrix:\n{cm}")
    
#     # Store the results
#     results[model_name] = {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1, 'confusion_matrix': cm}
    
#     # Keep track of the best model based on F1 score
#     if f1 > best_f1_score:
#         best_f1_score = f1
#         best_model = model_name

# print(f"\nThe best model is {best_model} with an F1 score of {best_f1_score:.2f}")


Results for CNN:
Accuracy: 71.89%
Precision: 0.78
Recall: 0.72
F1 Score: 0.71
Confusion Matrix:
[[126   6   0   0   5   0   0   0   1   7   2   2   0   0   0]
 [ 18 169   0   0  19   0   0   0   0   3   0   8   3   0   1]
 [  1   0 133   1   0   0   0   0   0   3   0   2   0   0  10]
 [  2   0   9 110  10   0   0   0   0   5   6   7   1   0   0]
 [  1   0   0   0  20   0   0   0   0   0   0   1   0   0   0]
 [  1   1   0  15   6 237   0   1   0   4   1  14  39   0   0]
 [ 14   2   3  15   0  19  34   5   0   7  16  20  12   0   3]
 [ 29   3  17  36   2   3   2 114   1  49   8   4   8   0  10]
 [ 20   0   4   0   4   1   0   4  36  32  13   3   0   6  19]
 [  7   2   4   2  23   2   0   0   0 177   1   5   4   8  30]
 [  6   0   0   2   4   0   0   0   0   0 208  25   2   0   4]
 [  0   0   0   3   8   0   0   0   0   0  19 142   0   0  38]
 [ 18   0   0   0   3   1   0   0   0   0  24   0 434   0   0]
 [  0   0   0   0   0   0   0   0   0   0   4   8   0  42   1]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0 238]]

Results for MobileNet:
Accuracy: 90.22%
Precision: 0.92
Recall: 0.90
F1 Score: 0.90
Confusion Matrix:
[[146   2   0   0   0   0   0   0   0   0   0   0   1   0   0]
 [  2 216   0   0   0   0   0   0   0   0   0   0   3   0   0]
 [  0   0 149   0   0   0   0   0   0   1   0   0   0   0   0]
 [  0   1   0 136   0   0   1  12   0   0   0   0   0   0   0]
 [  0   7   0   0  14   0   0   0   0   0   0   0   0   1   0]
 [  9   0   0   0   0 263   6   0   2   4   0   3  32   0   0]
 [  0   0   0   0   0   0 128   8   1  12   0   0   0   1   0]
 [  0   0   0   0   0   1   2 270  11   2   0   0   0   0   0]
 [  0   1   0   0   0   0   0   0 134   1   0   0   0   6   0]
 [  0   0   0   0   0   0   1   0   6 255   0   0   0   3   0]
 [  0   1   0   0   0   0   2   0   1   0 229   0   7  11   0]
 [  3   1   0   0   0   0   3   2   2  37  26 111   7  18   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0 480   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0  55   0]
 [  0   1   0   0   0   2   0   1   0  23   6   1   2   2 200]]

In [9]:
#CNN model
# CNN Model Function
from tensorflow.keras.models import Model
def build_cnn(input_shape, num_classes):
    model = models.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))# 32 filters - can be tweaked to incerase the variety of features learnt, 3x3 kernel, ReLU activation
    model.add(layers.MaxPooling2D((2, 2))) # (2,2) - Pooling sizes -reduces the spatial dimensions of the feature maps, retaining only the most prominent features (reducing the complexity).
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))# Increasing layers and filters (64)  we allow the network to capture a richer and more diverse set of features.
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))# Increasing layers and filters (128)  we allow the network to capture a richer and more diverse set of features.
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten and fully connected layers
    model.add(layers.Flatten()) # Flattens the 3D output from the last convolutional layer into a 1D array
    model.add(layers.Dense(128, activation='relu')) # This adds a fully connected layer with 128 neurons. Adjusting this number can affect how much information the model can learn
    model.add(layers.Dropout(0.5)) # This randomly drops 50% of the neurons during training to prevent overfitting.
    model.add(layers.Dense(num_classes, activation='softmax')) # This final layer outputs probabilities for each class using the softmax activation function. The number of neurons here should match your number of classes.( Thus num_classes)

    # Compile model
    model.compile(optimizer='adam',  # The Adam optimizer is used here, which adapts the learning rate during training. You can experiment with different optimizers like SGD, RMSprop, etc., and adjust their learning rates to improve performance.
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    return model

# Train CNN
print("Training CNN model...")

# Build the CNN model
cnn_model = build_cnn(input_shape=(224, 224, 3), num_classes=train_generator.num_classes)

# Train the CNN model
cnn_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=5  # Adjust this based on training performance
)


Training CNN model...


d:\Data_Science_Projects\env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5


d:\Data_Science_Projects\env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


451/451 ━━━━━━━━━━━━━━━━━━━━ 1267s 3s/step - accuracy: 0.1979 - loss: 2.5098 - val_accuracy: 0.3952 - val_loss: 1.8996
Epoch 2/5
  1/451 ━━━━━━━━━━━━━━━━━━━━ 14:09 2s/step - accuracy: 0.2812 - loss: 2.3144

C:\Users\wivan\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


451/451 ━━━━━━━━━━━━━━━━━━━━ 2s 623us/step - accuracy: 0.2812 - loss: 2.3144 - val_accuracy: 0.8125 - val_loss: 1.8009
Epoch 3/5
451/451 ━━━━━━━━━━━━━━━━━━━━ 1067s 2s/step - accuracy: 0.3782 - loss: 1.9438 - val_accuracy: 0.6038 - val_loss: 1.1971
Epoch 4/5
451/451 ━━━━━━━━━━━━━━━━━━━━ 3s 563us/step - accuracy: 0.4375 - loss: 1.5253 - val_accuracy: 1.0000 - val_loss: 0.5483
Epoch 5/5
451/451 ━━━━━━━━━━━━━━━━━━━━ 1089s 2s/step - accuracy: 0.5137 - loss: 1.4699 - val_accuracy: 0.7038 - val_loss: 0.8629

Evaluating CNN model...


NameError: name 'evaluate_model' is not defined

In [10]:
def evaluate_model(model, validation_generator):
    # Predict on validation data
    validation_generator.reset()  # Reset the generator to avoid any partial batches
    predictions = model.predict(validation_generator)
    
    # Get the predicted class indices
    y_pred = np.argmax(predictions, axis=1)
    
    # Get the true class labels
    y_true = validation_generator.classes
    
    # Calculate accuracy, precision, recall, and F1 score
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    # Calculate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    return accuracy, precision, recall, f1, cm


In [11]:
# Evaluate CNN model
print("\nEvaluating CNN model...")
cnn_accuracy, cnn_precision, cnn_recall, cnn_f1, cnn_cm = evaluate_model(cnn_model, validation_generator)


Evaluating CNN model...
97/97 ━━━━━━━━━━━━━━━━━━━━ 46s 468ms/step


d:\Data_Science_Projects\env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
# Print the metrics
print("\nCNN Model Evaluation Metrics:")
print(f"Accuracy: {cnn_accuracy * 100:.2f}%")
print(f"Precision: {cnn_precision:.2f}")
print(f"Recall: {cnn_recall:.2f}")
print(f"F1 Score: {cnn_f1:.2f}")
print(f"Confusion Matrix:\n{cnn_cm}")


CNN Model Evaluation Metrics:
Accuracy: 70.50%
Precision: 0.71
Recall: 0.70
F1 Score: 0.68
Confusion Matrix:
[[ 54  56   9   0   0   0   0   0   0  10  12   0   8   0   0]
 [  3 211   0   0   0   0   0   0   0   0   0   0   7   0   0]
 [  1   0 138   1   0   0   1   4   0   1   0   0   0   0   4]
 [  5   4  10  13   0   1   2  55   1   7  26   2   6   0  18]
 [  1  18   0   0   0   0   0   0   0   0   0   3   0   0   0]
 [  0   2   0   0   0 273   1  10   0   1   0   0  29   0   3]
 [ 16   5   7   0   0  18  48  17   0   1  13   6  18   1   0]
 [ 26   7  15   1   0  16  10 155   4   9  17   0   9   0  17]
 [ 12   8   1   0   0   3   0   2  62  23   5   0  24   0   2]
 [  8  30  14   0   0   6   0   4  20 150   4  11   7   4   7]
 [  2   1   0   0   0   2   0   0   1   3 190  27  19   2   4]
 [  5   3   0   0   0   1   0   0   0   0  44 134   6   1  16]
 [  1   0   0   0   0  10   0   0   0   0   3   0 466   0   0]
 [  0   0   0   0   0   0   0   0   0   2   0   0   1  52   0]
 [  0   

MobileNet Model

In [14]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# MobileNet Model Function
def build_mobilenet(input_shape, num_classes):
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # Add global average pooling and dense layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=x)
    
    # Freeze base layers
    for layer in base_model.layers:
        layer.trainable = False

    # Compile model
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    return model

# Train MobileNet
print("Training MobileNet model...")

# Build the MobileNet model
mobilenet_model = build_mobilenet(input_shape=(224, 224, 3), num_classes=train_generator.num_classes)

# Train the MobileNet model
mobilenet_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=5  # Adjust this based on training performance
)

# Evaluate MobileNet model
print("\nEvaluating MobileNet model...")
mobilenet_accuracy, mobilenet_precision, mobilenet_recall, mobilenet_f1, mobilenet_cm = evaluate_model(mobilenet_model, validation_generator)


Training MobileNet model...
Epoch 1/5
451/451 ━━━━━━━━━━━━━━━━━━━━ 728s 2s/step - accuracy: 0.6779 - loss: 1.0451 - val_accuracy: 0.8444 - val_loss: 0.4773
Epoch 2/5
  1/451 ━━━━━━━━━━━━━━━━━━━━ 7:08 953ms/step - accuracy: 0.8750 - loss: 0.3423

C:\Users\wivan\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


451/451 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8750 - loss: 0.3423 - val_accuracy: 0.5000 - val_loss: 0.9944
Epoch 3/5
451/451 ━━━━━━━━━━━━━━━━━━━━ 611s 1s/step - accuracy: 0.8890 - loss: 0.3462 - val_accuracy: 0.8968 - val_loss: 0.3179
Epoch 4/5
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 967us/step - accuracy: 0.8438 - loss: 0.3541 - val_accuracy: 0.8125 - val_loss: 0.3224
Epoch 5/5
451/451 ━━━━━━━━━━━━━━━━━━━━ 717s 2s/step - accuracy: 0.9057 - loss: 0.2687 - val_accuracy: 0.8926 - val_loss: 0.3119

Evaluating MobileNet model...
97/97 ━━━━━━━━━━━━━━━━━━━━ 91s 925ms/step


In [15]:
# CNN Results
print("\nResults for CNN:")
print(f"Accuracy: {cnn_accuracy * 100:.2f}%")
print(f"Precision: {cnn_precision:.2f}")
print(f"Recall: {cnn_recall:.2f}")
print(f"F1 Score: {cnn_f1:.2f}")
print(f"Confusion Matrix:\n{cnn_cm}")

# MobileNet Results
print("\nResults for MobileNet:")
print(f"Accuracy: {mobilenet_accuracy * 100:.2f}%")
print(f"Precision: {mobilenet_precision:.2f}")
print(f"Recall: {mobilenet_recall:.2f}")
print(f"F1 Score: {mobilenet_f1:.2f}")
print(f"Confusion Matrix:\n{mobilenet_cm}")



Results for CNN:
Accuracy: 70.50%
Precision: 0.71
Recall: 0.70
F1 Score: 0.68
Confusion Matrix:
[[ 54  56   9   0   0   0   0   0   0  10  12   0   8   0   0]
 [  3 211   0   0   0   0   0   0   0   0   0   0   7   0   0]
 [  1   0 138   1   0   0   1   4   0   1   0   0   0   0   4]
 [  5   4  10  13   0   1   2  55   1   7  26   2   6   0  18]
 [  1  18   0   0   0   0   0   0   0   0   0   3   0   0   0]
 [  0   2   0   0   0 273   1  10   0   1   0   0  29   0   3]
 [ 16   5   7   0   0  18  48  17   0   1  13   6  18   1   0]
 [ 26   7  15   1   0  16  10 155   4   9  17   0   9   0  17]
 [ 12   8   1   0   0   3   0   2  62  23   5   0  24   0   2]
 [  8  30  14   0   0   6   0   4  20 150   4  11   7   4   7]
 [  2   1   0   0   0   2   0   0   1   3 190  27  19   2   4]
 [  5   3   0   0   0   1   0   0   0   0  44 134   6   1  16]
 [  1   0   0   0   0  10   0   0   0   0   3   0 466   0   0]
 [  0   0   0   0   0   0   0   0   0   2   0   0   1  52   0]
 [  0   0   0   0   0